# Building an **AI Research Assistant** with the OpenAI Agents SDK

This notebook provides a reference patterns for implementing a multi‑agent AI Research Assistant that can plan, search, curate, and draft high‑quality reports with citations.

While the Deep Research feature is avaialble in ChatGPT, however, individual and companies may want to implement their own API based solution for a more finegrained control over the output.

With support for Agents, and built-in tools such as Code Interpreter, Web Search, and File Search, - Responses API makes building your own Research Assistant fast and easy. 

## Table of Contents
1. [Overview](#overview)
2. [Solution Workflow](#workflow)
3. [High‑Level Architecture](#architecture)
4. [Agent Definitions (Pseudo Code)](#agents)
    * Research Planning Agent
    * Web Search Agent
    * Knowledge Assistant Agent
    * Report Creation Agent
    * Data Analysis Agent (optional)
    * Image‑Gen Agent (optional)
5. [Guardrails & Best Practices](#best-practices)
6. [Risks & Mitigation](#risks)

### 1 — Overview <a id='overview'></a>
The AI Research Assistant helps drives better research quality and faster turnaround for knowledge content.

1. **Performs autonomous Internet research** to gather the most recent sources.
2. **Incorporates internal data sources** such as a Company's proprietery knowledge sources. 
3. **Reduces analyst effort from days to minutes** by automating search, curation and first‑draft writing.
4. **Produces draft reports with citations** and built‑in hallucination detection.

### 2 — Solution Workflow <a id='workflow'></a>
The typical workflow consists of five orchestrated steps: 

| Step | Purpose | Model |
|------|---------|-------|
| **Query Expansion** | Draft multi‑facet prompts / hypotheses | `gpt‑4o` |
| **Search‑Term Generation** | Expand/clean user query into rich keyword list | `gpt‑4o` |
| **Conduct Research** | Run web & internal searches, rank & summarise results | `gpt‑4o` + tools |
| **Draft Report** | Produce first narrative with reasoning & inline citations | `o1` / `gpt‑4o` |
| **Report Expansion** | Polish formatting, add charts / images / appendix | `gpt‑4o` + tools |

### 3 — High‑Level Architecture <a id='architecture'></a>
The following diagram groups agents and tools:

* **Research Planning Agent** – interprets the user request and produces a research plan/agenda.
* **Knowledge Assistant Agent** – orchestrates parallel web & file searches via built‑in tools, curates short‑term memory.
* **Web Search Agent(s)** – perform Internet queries, deduplicate, rank and summarise pages.
* **Report Creation Agent** – consumes curated corpus and drafts the structured report.
* **(Optional) Data Analysis Agent** – executes code for numeric/CSV analyses via the Code Interpreter tool.
* **(Optional) Image‑Gen Agent** – generates illustrative figures.

Input/output guardrails wrap user prompts and final content for policy, safety and citation checks.

### 4 — Pre-requisites <a id='pre-requisites'></a>

Create a virual environment  

Install dependencies 

In [1]:
%pip install openai openai-agents --quiet

Note: you may need to restart the kernel to use updated packages.


### 5 — Agents (Pseudo Code) <a id='agents'></a>
Below are skeletal class definitions illustrating how each agent’s policy and tool‑usage might look.

#### Step 1 - Query Expansion

In [ ]:
from agents import Agent, Runner

query_expansion_agent = Agent(
    name="Query Expansion Agent",
    instructions="""You are a helpful agent who is given a research prompt from the user as input.  
        Your task is to expand the prompt into a more complete and actionable research prompt. Do not write the research 
        paper, just improve the prompt in about one paragraph. Only respond with the expanded prompt no qualifiers.""",
    tools=[],
    model="gpt-4o",    
)

result = await Runner.run(query_expansion_agent, "Draft a research report on latest trends in persona auto insurance in the US")

expanded_prompt = result.final_output 

print(expanded_prompt)

Draft a comprehensive research report analyzing the evolution and impact of artificial intelligence (AI) over the past five years. This report should investigate key trends that have emerged during this period, including advancements in machine learning models like GPT and BERT, the rise of AI in industries such as healthcare, finance, and autonomous vehicles, and the ethical considerations surrounding AI development and implementation. Delve into how these trends have influenced technological growth, business strategies, and regulatory measures globally. Evaluate the societal and economic implications of these advancements and provide insights into future directions AI might take. Use a variety of sources, including scholarly articles, industry reports, and expert interviews, to support your analysis and conclusions.


#### Step 2 - Web Search Terms 

Generate the web search terms. You can customize the number of search terms generated to a give level of depth. 

In [13]:
from pydantic import BaseModel

class SearchTerms(BaseModel):
    """Structured output model for search-terms suggestions."""
    Search_Queries: list[str]


search_terms_agent = Agent(
    name="Search Terms Agent",
    instructions="""You are a helpful agent assigned a research task. Your job is to provide the top 
        5 Search Queries relevant to the given topic in this year (2025). The output should be in JSON format.

        Example format provided below:
        <START OF EXAMPLE>
        {
        "Search_Queries": [
            "Top ranked auto insurance companies US 2025 by market capitalization",
            "Geico rates and comparison with other auto insurance companies",
            "Insurance premiums of top ranked companies in the US in 2025", 
            "Total cost of insuring autos in US 2025", 
            "Top customer service feedback for auto insurance in 2025"
        ]
        }
        </END OF EXAMPLE>
        """,
    tools=[],
    model="gpt-4o",     
    output_type=SearchTerms,
)

result = await Runner.run(search_terms_agent, expanded_prompt)

search_terms_raw = result.final_output

print(search_terms_raw)


for query in enumerate(search_terms_raw.Search_Queries):
    print(f"{query}")

Search_Queries=['Evolution of AI technology 2020-2025', 'Impact of machine learning models GPT and BERT on industries', 'AI advancements in healthcare and finance 2025', 'Ethics and AI development 2025', 'Future directions for artificial intelligence in various sectors']
(0, 'Evolution of AI technology 2020-2025')
(1, 'Impact of machine learning models GPT and BERT on industries')
(2, 'AI advancements in healthcare and finance 2025')
(3, 'Ethics and AI development 2025')
(4, 'Future directions for artificial intelligence in various sectors')


In [ ]:
class KnowledgeAssistantAgent:
    """Curates short‑term memory of research snippets."""
    def run(self, web_snippets, file_snippets):
        corpus = web_snippets + file_snippets
        # Vector‑embed & cluster (pseudo)
        # ...
        # Return pruned, deduplicated corpus
        return corpus[:50]  # top‑N

In [ ]:
class ReportCreationAgent:
    """Drafts the first complete report with citations."""
    def run(self, curated_corpus, outline):
        report = client.chat(
            model='gpt-4o',
            system_prompt='Write a research report following the outline. Cite sources in IEEE style.',
            user_prompt=str({'outline': outline, 'corpus': curated_corpus}),
        )
        return report.text

In [ ]:
# --- Orchestration skeleton ---
def generate_research_paper(topic: str):
    plan = ResearchPlanningAgent().run(topic)

    web_results = WebSearchAgent().run(plan['search_terms'])
    # TODO: file_results via file_search if internal corpus available
    file_results = []

    curated = KnowledgeAssistantAgent().run(web_results, file_results)
    draft = ReportCreationAgent().run(curated, plan['outline'])
    return draft


### 5 — Guardrails & Best Practices <a id='best-practices'></a>
* **Crawl → Walk → Run**: start with a single agent, then expand into a swarm. 
* **Expose intermediate reasoning** (“show the math”) to build user trust. 
* **Parameterise UX** so analysts can tweak report format and source mix. 
* **Native OpenAI tools first** (web browsing, file ingestion) before reinventing low‑level retrieval. 

### 6 — Risks & Mitigation <a id='risks'></a>
| Pitfall | Mitigation |
|---------|------------|
| Scope‑creep & endless roadmap | Narrow MVP & SMART milestones | fileciteturn1file4L23-L24 |
| Hallucinations & weak guardrails | Golden‑set evals, RAG with citation checks | fileciteturn1file4L25-L26 |
| Run‑away infra costs | Cost curve modelling; efficient models + autoscaling | fileciteturn1file4L27-L28 |
| Talent gaps | Upskill & leverage Agents SDK to offload core reasoning | fileciteturn1file4L29-L30 |